In [ ]:
import requests
import copy
import pandas as pd
import itchat
from itchat.content import *
import time
import datetime 
import re
import json
import pymongo
# from datetime import datetime
import china_region
import pysnooper
from datetime import timedelta

In [ ]:
myclient = pymongo.MongoClient(
    host='localhost',
    port=27017,
    serverSelectionTimeoutMS=10)
# myclient.server_info()  # 查看数据库信息，在这里用于是否连接数据的测试
wechat_helper_db = myclient["wechat_helper"]
mocov_num_db = wechat_helper_db["mocov_num"]
mocov_new_db = wechat_helper_db["mocov_new"]


In [ ]:
clean_at_compile = r'(@.*?\s{1,}|\s*$|^\s*)'
conx_new_key = "疫情新闻"
conx_news_city_complie = rf"^{conx_new_key}\s*([\u4e00-\u9fa5]*)\s*(\+*)\s*$"
conx_news_id_complie = rf"^{conx_new_key}\s*(\d+)$\s*$"
time_delta_compile = r'(?:(\d+)\s+days?,)?\s*(\d{1,2}):(\d{1,2}):\d{1,2}'  # 项目登录时间间隔
conx_num_city_compile = r'\s*([\u4e00-\u9fa5]+)\s*(\+*)\s*$'  # 项目登录时间间隔

cache_conx_num_dict = {}
cache_conx_city_dict = {}

In [ ]:
# with open('数据.txt','r',encoding='utf-8') as f:
#     all_datas = f.readlines()
    

In [ ]:
def update_conx_news_info():
    """
    自动更新疫情新闻
    :return:
    """
    try:
        url2 = "https://lab.isaaclin.cn/nCoV/api/news?num=all"
        resp2 = requests.get(url2)

        if not resp2.status_code == 200:
            return None

        data2 = resp2.json()
        # 获取数据
        newsest_data = data2['results']
        if not newsest_data:
            return None
        # 对数据进行排序
        newsest_data_sort = sorted(newsest_data, key=lambda x: x['pubDate'], reverse=False)

        # 取出到最后一个的数据
        last_data = mocov_new_db.find_one({}, sort=[('id', -1)])
        if last_data is not None:
            last_id = last_data['id']
            last_pub = last_data['pubDate']
        else:
            last_id = 0
            last_pub = 0
        # 前一次新闻总数量

        # 保存数据
        is_update = False
        for item in newsest_data_sort:
            pubDate = item['pubDate']
            # 对时间进行判断，如果小于最后保存时间，则跳过。
            if pubDate < last_pub:
                continue

            # 判断数据是否存在数据库中
            count = mocov_new_db.count_documents(item)
            if count == 0:
                is_update = True
                last_id = last_id + 1  # id + 1
                pub_time_str = datetime.datetime.fromtimestamp(item['pubDate'] / 1000).strftime("%Y-%m-%d %H:%M:%S")
                item['pub_time_str'] = pub_time_str
                item['id'] = last_id
                # 保存数据
                mocov_new_db.insert_one(item)

        if is_update:
            # 从数据库把所有数据取出来,作为缓存处理
            db_data = list(mocov_new_db.find())
            global db_conx_news_data_dict
            global cache_conx_city_dict
            db_conx_news_data_dict = {i['id']: i for i in db_data}
            # 清空查询缓存数据
            cache_conx_city_dict.clear()

    except Exception as exception:
        print(str(exception))
    return None

update_conx_news_info()

In [ ]:
def update_conx_num_info():
    """
    获取疫情数据
    :return:
    """
    try:
        url = 'http://api.tianapi.com/txapi/ncovcity/index?key=064b478e2175e6dd27931cc93d6292cf'
        resp = requests.get(url)
        if resp.status_code == 200:
            data = resp.json()
            all_data = []
            last_time = datetime.datetime.now()

            summary_item = {
                "provinceName": "全国",
                "cityName": "全国",
                "confirmedCount": 0,
                "suspectedCount": 0,
                "curedCount": 0,
                'deadCount': 0,
                'level': 2,
                'last_time': last_time
            }

            for item in data['newslist']:
                single_item = copy.deepcopy(item)
                single_item['cityName'] = item['provinceShortName']
                del single_item['comment']
                del single_item['cities']
                del single_item['provinceShortName']

                summary_item['confirmedCount'] = summary_item['confirmedCount'] + item['confirmedCount']
                summary_item['suspectedCount'] = summary_item['suspectedCount'] + item['suspectedCount']
                summary_item['curedCount'] = summary_item['curedCount'] + item['curedCount']
                summary_item['deadCount'] = summary_item['deadCount'] + item['deadCount']

                single_item['level'] = 1
                single_item['last_time'] = last_time
                all_data.append(single_item)

                provinceName = item['provinceName']
                for cn in item.get('cities'):
                    cn['provinceName'] = provinceName
                    cn['level'] = 0
                    cn['last_time'] = last_time
                    all_data.append(cn)
            all_data.append(summary_item)

            # 保存到数据库
            for item in all_data:
                key = {
                    'provinceName': item['provinceName'],
                    'cityName': item['cityName'],
                }
                mocov_num_db.update_one(key, {"$set": item}, upsert=True)

            # 清除确珍人数缓存
            cache_conx_num_dict.clear()


    except Exception as exception:
        print(str(exception))
    return None

update_conx_num_info()

In [ ]:
# 查询确珍人数
@pysnooper.snoop()
def get_ncox_num(provinceName, plus=False):
    return_msg = ''
    
    provinceName = provinceName[:-1] if len(provinceName)>2 and provinceName[-1] in '省市区县旗盟州区港门台尔' else provinceName
    key = {
        '$or':[
            {'provinceName':{'$regex': provinceName}},
            {'cityName':{'$regex': provinceName},"level":0},
        ]
    }
    if mocov_num_db.count_documents(key) == 0:
        city_info = china_region.search(provinceName)
        if not city_info:
            return

        province_name = city_info['city']
        province_name = province_name[:-1] if len(province_name)>2 and province_name[-1] in '省市区县旗盟州区港门台尔' else province_name
        key = {
            '$or':[
                {'provinceName':{'$regex': province_name}},
                {'cityName':{'$regex': province_name},"level":0},
            ]
        }
        if mocov_num_db.count_documents(key) == 0:
            return

    s_data = mocov_num_db.find_one(key)

    return_msg = f"{s_data['cityName']} 确认{s_data['confirmedCount']}例"
    
    # 全省或者全国的数据     
    if plus and s_data.get('level', 0) > 0:
        if provinceName == '全国':
            plus_city_key = {
                'level': 1
                }
        else:
            plus_city_key = {
                'provinceName':{'$regex': provinceName},
                'level': 0
            }
        count = mocov_num_db.count_documents(plus_city_key)
        plus_data = list(mocov_num_db.find(plus_city_key,sort=[('confirmedCount',-1)]))
        plus_msg = '\n'.join(f"{item['cityName']} 确认{item['confirmedCount']}例" for item in plus_data)
        return_msg = f'{return_msg}\n-----------\n{plus_msg}'
#     print(return_msg)
    return return_msg
                             

city_name = '济南'   
plus = True
get_ncox_num(city_name,True)

In [ ]:
# @pysnooper.snoop()
# 新闻显示的日期
def novx_time_delta(novx_time):
    # 分钟前，小时前 ，昨天 09:40 , 前天 06：11 ，02-12 
#     time_delta = timedelta(seconds=novx_time)
    msg = ''
    
    pub_time = datetime.datetime.fromtimestamp(novx_time//1000)
    now_time = datetime.datetime.now()
    time_delta = now_time - pub_time
    
    days = time_delta.days
    seconds = time_delta.seconds
    if days == 0:
        if seconds < 60*60:
            msg = f"{seconds//60}分钟前"
#             print(msg)
            return msg
        elif seconds < 24*60*60:
            msg = f"{seconds//(60*60)}小时前"
#             print(msg)
            return msg
    elif days == 1:
        msg = f"昨天 {pub_time.strftime('%H:%M')}"
        return msg
    elif days == 2:
        msg = f"前天 {pub_time.strftime('%H:%M')}"
        return msg
    elif days == 3:
        msg = f"三天前"
        return msg
    elif days > 3:
        msg = f"{pub_time.strftime('%m-%d')}"
        return msg
    return msg

novx_time = find_province('',limit=1)[-1]['pubDate']
novx_time_delta(novx_time)

#       return ""

In [ ]:
# 根据省份查询数据
# @pysnooper.snoop()
def get_novx_province_news(provinceName,limit=10):
    
    if len(provinceName) == 1:
        return None
    
    key = {"provinceName" : {'$regex': provinceName}}
    count = mocov_new_db.count_documents(key)

    if count == 0:
        # 去除最后一个字
        if len(provinceName) > 2 and provinceName[-1] in '省市区县旗盟州区港门台尔':
            provinceName = provinceName[:-2]
            key = {"provinceName" : {'$regex': provinceName}}
            count = mocov_new_db.count_documents(key)
               
           
    # 有可能是市或者县城。             
    if count == 0:
        # 通过县城反推出市的名称        
        city_info = china_region.search(provinceName)
        if city_info:
            province = city_info['province']
            if province is None:
                return None
            key = {"provinceName" : {'$regex': province}}
            count = mocov_new_db.count_documents(key)
            if count == 0:
                return None  
    return_data = mocov_new_db.find(key,sort=[('id',-1)]).limit(limit)
    if return_data is not None:
        return list(return_data)
    return None
    

provinceName = "湖北"
provinceName = ""
data = get_novx_province_news(provinceName)
if data is not None:
    msg_list = [f"{novx_time_delta(i['pubDate'])} {i['title']}(ID {i['id']})" for i in data]
    msg = '\n'.join(msg_list)
    print(msg)

In [ ]:
get_novx_province_news(provinceName)[-1]

In [ ]:
key_name = '桂林'
key = {"$or" : [{'title':{'$regex': key_name}},{'summary':{'$regex': key_name}}]}
list(mocov_new_db.find(key).l n imit(5))

In [ ]:
# 疫情新闻 广西
# 疫情新闻 1024
# 广西

In [ ]:
@itchat.msg_register([TEXT], isGroupChat=True)
def group_note_msg(msg):
    """
    处理群组里的-热门消息
    """
    log(msg)
    

In [ ]:
def log(msg):
    if not msg.isAt:
        return
    text = msg['Text']  # 发送到群里的消息。
    msg_group_uid = msg['User']['UserName']  # 群uid
    yy(text, msg_group_uid, flag=True)

In [ ]:
@pysnooper.snoop()
def yy(text,msg_group_uid='',flag=False):
    # 清理艾特与空格      
    text = re.sub(clean_at_compile, '', text)
#     print(text)
    if len(text) < 2:
        return 
    
    _ids = re.findall(conx_news_id_complie, text)
    if _ids:
        _id = int(_ids[0])
        if _id in db_conx_news_data_dict:
            item = db_conx_news_data_dict.get(_id,None)
            if item:
                msg = f"标题：{item['title']}\n副标题：{item['summary'].strip()}\n来源：{item['infoSource']}\n省份：{item['provinceName']}\n发布时间：{item['pub_time_str']}\n链接地址：{item['sourceUrl']}"
                if flag:
                    itchat.send(msg, msg_group_uid)
                else:
                    print(msg)
                return 
        return 
    
    citys = re.findall(conx_news_city_complie, text)
    if citys:
         # 从缓存中取出数据    
        if text in cache_conx_city_dict:                 
            msg = cache_conx_city_dict.get(text,None)
#             print(msg)
            itchat.send(msg, msg_group_uid)
            return 
        
        city, plus = citys[0][0],citys[0][1]
        if city in ('最新'," ",'全国','中国'):
            city = ''
            
        limit = (len(plus) + 1) * 10
        data = get_novx_province_news(city,limit=limit)
        if data:
            msg_list = [f"{novx_time_delta(i['pubDate'])} {i['title']}(ID:{i['id']})" for i in data]  
            msg = '\n'.join(msg_list)
            cache_conx_city_dict[text] = msg
            if flag:
                itchat.send(msg, msg_group_uid)
            else:
                print(msg)
            return
        return
    
    
    conx_nums = re.findall(conx_num_city_compile, text)
    if conx_nums:
        # 从缓存中取出数据         
        if text in cache_conx_num_dict:                 
            msg = cache_conx_num_dict.get(text,None)
#             print(msg)
            itchat.send(msg, msg_group_uid)
            return 
    
        city_name, plus = conx_nums[0]
        if city_name in ('最新'," ",'全国','中国'):
            city_name = '全国'
        is_plus = True if len(plus) > 0 else False
        # 查询请求       
        msg = get_ncox_num(city_name, is_plus)
        if msg is not None and len(msg) > 1:
            cache_conx_num_dict[text] = msg         
            if flag:
                itchat.send(msg, msg_group_uid)
            else:
                print(msg)

        return 
# cache_conx_num_dict.clear()
text = "疫情新闻+"
yy(text)      

In [83]:
itchat.logout()
itchat.auto_login()
itchat.run(blockThread=False)

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 贴心小秘书
Start auto replying.
